In [2]:
pwd

'/root/CodeCommit/eks-ml-pipeline'

In [3]:
!pip uninstall -y msspackages

Keyring is skipped due to an exception: 'keyring.backends'


In [4]:
!pip install /root/CodeCommit/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/CodeCommit/msspackages/dist/msspackages-0.0.7-py3-none-any.whl
  Using cached pyspark-3.3.1-py2.py3-none-any.whl
  Using cached configparser-5.3.0-py3-none-any.whl (19 kB)
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)


In [5]:
from msspackages import setup_runner
setup_runner()

b"Get:1 http://deb.debian.org/debian buster InRelease [122 kB]\nGet:2 http://security.debian.org/debian-security buster/updates InRelease [34.8 kB]\nGet:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]\nGet:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [411 kB]\nGet:5 http://deb.debian.org/debian buster/main amd64 Packages [7909 kB]\nGet:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]\nFetched 8542 kB in 1s (6205 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n62 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following NEW packages will be installed:\n  sudo\n0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.\nNeed to get 1244 kB of archives.\nAfter this operation, 3883 kB of additional disk space will be used.\nGet:1 http://security.debian.org/

In [6]:
!python3 -m pip install -r requirements.txt

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.11.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_io_gcs_filesystem-0.28.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached libclang-14.0.6-py2.py3-none-manylinux2010_x86_64.whl (14.1 MB)
  Using cached grpcio-1.51.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached protobuf-3.19.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB

In [2]:
%%time
from eks_ml_pipeline import pod_autoencoder_ad_preprocessing, pod_autoencoder_ad_feature_engineering, pod_autoencoder_train_test_split

#pre processing
pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_autoencoder_ad","11-30-2022","2022","10","10","10","128gb")

CPU times: user 3.03 s, sys: 1.7 s, total: 4.72 s
Wall time: 12.7 s


In [3]:
pod_features_data

,feature_name,feature_description,feature_datatype,feature_type,feature_group_name,feature_group_description,model_type,problem_type,created_by,created_date,model_parameters
0,pod_cpu_utilization,The percentage of CPU cores utilized on a per ...,float,existing,pod_autoencoder_ad,general features to track the infrastructure u...,autoencoder,anomaly detection,"Madhu, Evgeniya",11-30-2022,"{'time_steps': 12, 'batch_size': 6, 'sample_mu..."
1,pod_memory_utilization,The percentage of memory currently being used ...,float,existing,pod_autoencoder_ad,general features to track the infrastructure u...,autoencoder,anomaly detection,"Madhu, Evgeniya",11-30-2022,"{'time_steps': 12, 'batch_size': 6, 'sample_mu..."
2,pod_network_total_bytes,The amount of all network recreived and transf...,float,existing,pod_autoencoder_ad,general features to track the infrastructure u...,autoencoder,anomaly detection,"Madhu, Evgeniya",11-30-2022,"{'time_steps': 12, 'batch_size': 6, 'sample_mu..."


In [4]:
#test, train split
pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data)


In [5]:
#Train data feature engineering
pod_training_data, pod_training_tensor = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_train_data)

Finished with sample #0
Finished with sample #1
Finished with sample #2
Finished with sample #3
Finished with sample #4
Finished with sample #5


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import numpy as np
np.savetxt('pod_training_tensor.txt', pod_training_tensor.reshape(pod_training_tensor.shape[0], -1))

In [13]:
%%time
from eks_ml_pipeline import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split

#pre processing
node_features_data, node_processed_data = node_autoencoder_ad_preprocessing("node_autoencoder_ad","11-21-2022","2022","10","10","10","128gb")



CPU times: user 53.6 ms, sys: 21.2 ms, total: 74.8 ms
Wall time: 12.4 s


In [ ]:
#test, train split
node_train_data, node_test_data = node_autoencoder_train_test_split(node_processed_data)

#Train data feature engineering
node_training_data, node_training_tensor = node_autoencoder_ad_feature_engineering(node_features_data, node_train_data)
print(node_training_data.show(truncate=False))
print(node_training_tensor.shape)

#Test data feature engineering
# node_testing_data, node_testing_tensor = node_autoencoder_ad_feature_engineering(node_features_data, node_test_data)
# print(node_testing_data.show(truncate=False))
# print(node_testing_tensor.shape)

i-0c4195b737f3ea65b
i-0c16de54e70456edb
i-0f7c2847250d4bf17
i-00f549b1977d71421
i-069f59f3cb835a468
i-089b266ea703b601b
i-083a68dd2c550b41f
i-016563cb8309a7094
i-051fd5738fb2c6ea1
i-0196f89befc2119b0
i-0b4e5a0f008c8710b
i-0e38bd7b4f6afd740
i-055d3b9ea035c9f49
i-0d8a08d4c37a89dd4
i-07e26d88fc0415c88
i-0c0f5401459e929d9
i-0613bdb45fb9738a8
i-049a23e425880667d
i-04a10e30a5989d602
i-0cbb588697c71c996
i-05e69fda46bdcc13c
i-005233f00f907ec3e
i-0bcd4ea3fc9cd87a7
i-016e870187fd08282
i-0ea37c2a5d64f001e
i-0d5cd6af4b8d70bcc
i-0b9a635fd77e60086
i-05f48056e7c8cc2f8
i-0b06d7929734611d0
i-02ffb539acc0cbf94
i-0c19364198e2ed5d1
i-0b4e5a0f008c8710b
i-0c63afd423cb8fa97
i-09e89aaa35264cd47
i-03257ddb394441bbc
i-0f72b3214815c581e
i-089b266ea703b601b
i-06d98ce4e70468c94
i-0786482c940838bec
i-08b01870c71691ba3
i-0a0a7354e7757e1e8
i-03fa1ba38efb4c9c2
i-08a4ed5bf3f1e2238
i-00f549b1977d71421
i-03a03e9e832962ba5
i-0d5795fd58c96e20d
i-0ee5ad2215c7ca388
i-0e3bca18935c2e996
i-0e7832552c9f31487
i-00f549b1977d71421


In [ ]:
%%time
from eks_ml_pipeline import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split

#pre processing
container_features_data, container_processed_data = container_autoencoder_ad_preprocessing("container_autoencoder_ad","11-21-2022","2022","10","10","10","128gb")

#test, train split
container_train_data, container_test_data = container_autoencoder_train_test_split(container_processed_data)

#Train data feature engineering
container_training_data, container_training_tensor = container_autoencoder_ad_feature_engineering(container_features_data, container_train_data)
print(container_training_data.show(truncate=False))
print(container_training_tensor.shape)

#Test data feature engineering
# container_testing_data, container_testing_tensor = container_autoencoder_ad_feature_engineering(container_features_data, container_test_data)
# print(node_testing_data.show(truncate=False))
# print(container_testing_tensor.shape)